In [1]:
import numpy as np
import pandas as pd
import json
import lxml.html as lh
import requests
import urllib.request
import bs4 as bs
from sklearn.cluster import KMeans
import folium
print('Libraries imported')

Libraries imported


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Getting the html code from wikepedia 
def scrape_table(name, cols):
    page = urllib.request.urlopen(url).read()
    soup= bs.BeautifulSoup(page,'lxml')
    table = soup.find('table', class_=name)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all('th')]
    data = [[td.findAll(text=True)[0].strip() for td in tr.find_all('td')] for tr in table.find_all('tr')]
    data =[row for row in data if len(row) == cols]
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

def scrape_table_lxml(xPATH, cols):
    page= requests.get(url)
    doc =lh.fromstring(page.content)
    table_content = doc.xpath(xPath)
    for table in table_content: 
        headers=[th.text_content().strip() for th in table.xpath('//th')]
        headers= headers[0:3]
        data = [[td.text_content().strip() for td in tr.xpath('td')]
               for tr in table.xpath('//tbody/tr')]
        data = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data, columns = headers)
        return raw_df

In [3]:
raw_TorontoPostalCodes = scrape_table('wikitable',3)
raw_TorontoPostalCodes.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [4]:
#Data cleaning
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]

In [11]:
#Sort and reset Index 
TorontoPostalCodes = TorontoPostalCodes.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

KeyError: ignored

In [6]:
#If a cell has a borough but not an assigned neighbourhood then the neighbourhood will be the same as the borough

TorontoPostalCodes.loc[TorontoPostalCodes['Neighbourhood'] == 'Not assigned',['Neighbourhood']] = TorontoPostalCodes['Borough']

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'Queen\'s Park']
#print('DEBUG:',check_unassigned_post_state_sample) ; # Print sample borough problem post state

In [8]:
 -----------------------------------------------------
# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice 
# and has two neighborhoods: Harbourfront and Regent Park. 
# These two rows will be combined into one row with the neighborhoods separated with a comma.
# -----------------------------------------------------

TorontoPostalCodes = TorontoPostalCodes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

SyntaxError: ignored

In [9]:
TorontoPostalCodes

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
TorontoPostalCodes.to_csv('Toronto.TASK_1_df.csv',index = False)